In [ ]:
import random
import cv2
import torchvision
import numpy
import torch
from matplotlib import pyplot as plt
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.utils.visualizer import Visualizer
from detectron2.engine import DefaultTrainer, default_argument_parser, default_setup, hooks, launch
from detectron2.data.datasets import register_coco_instances

register_coco_instances(
    "phone_train",
    {},
    "data/phone/annotations/train.json",
    "data/phone/train"
)
register_coco_instances(
    "phone_val",
    {},
    "data/phone/annotations/val.json",
    "data/phone/val"
)

cfg = get_cfg()
cfg.merge_from_file("phone.yaml")
cfg.MODEL.WEIGHTS = "output/model_final.pth"

dataset_dicts = DatasetCatalog.get("phone_val")
meta = MetadataCatalog.get("phone_val")

In [ ]:
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.4
predictor = DefaultPredictor(cfg)
for d in random.sample(dataset_dicts, 10):
    img = cv2.imread(d["file_name"])
    out = predictor(img)
    visualizer = Visualizer(img[:, :, ::-1], metadata=meta, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    visimg = vis.get_image()[:, :, ::-1]
    # 添加这一行
    visualizer = Visualizer(img[:, :, ::-1], metadata=meta, scale=0.5)
    pre = visualizer.draw_instance_predictions(out["instances"].to("cpu"))
    preimg = pre.get_image()[:, :, ::-1]
    
    plt.figure()
    img = cv2.resize(img, (64, 64), interpolation = cv2.INTER_AREA)
    imgs = numpy.stack((img, visimg, preimg))
    imgs = imgs.transpose((0,3,1,2))
    imgs = torch.from_numpy(imgs)
    imgs = torchvision.utils.make_grid(imgs).numpy().transpose((1, 2, 0))
    plt.imshow(imgs)

In [ ]:
xa = []
cra = []
fpra = []
fnra = []

minidata = dataset_dicts

for th in numpy.arange(0.1, 0.95, 0.1):
    total = 0
    tp = 0
    fp = 0
    fn = 0
    tn = 0
    
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = float(th)
    predictor = DefaultPredictor(cfg)
    
    for d in minidata:
        img = cv2.imread(d["file_name"])
        out = predictor(img)
        t=len(d['annotations']) == 0
        y=len(out["instances"]) == 0
        if t == y == True:
            tp += 1
        elif t == y == False:
            tn += 1
        elif t == True:
            fn += 1
        else:
            fp += 1
        total += 1
        
    cr = (tp+tn)/total
    fpr = fp/(fp+tn)
    fnr = fn/(fn+tp)
    
    print("thre: %.2f, correct: %.2f, fpr: %.2f, fnr: %.2f" % (th, cr, fpr, fnr))
    
    xa.append(th)
    cra.append(cr)
    fpra.append(fpr)
    fnra.append(fnr)

In [ ]:
plt.figure()
plt.plot(xa,cra,label='Precision ')
plt.plot(xa,fpra,label='FPR')
plt.plot(xa,fnra,label='FNR')
plt.legend()
plt.show()